In [43]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow

SCOPES = ['https://www.googleapis.com/auth/generative-language.retriever']

def load_creds():
    """Converts `client_secret.json` to a credential object.

    This function caches the generated tokens to minimize the use of the
    consent screen.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secret.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return creds
creds = load_creds()

In [44]:
categories = '''เรียกรับสินบน ให้/ขอให้ หรือรับว่าจะให้ทรัพย์สินหรือประโยชน์อื่นใดแก่เจ้าหน้าที่ของรัฐ
จัดซื้อจัดจ้าง
ยักยอก/เบียดบังเงินหรือทรัพย์สินของทางราชการ 
ออกเอกสารสิทธิ
การบริหารงานบุคคล (การบรรจุ/แต่งตั้ง/เลื่อนตำแหน่ง/โยกย้าย/ลงโทษวินัย) 
ทุจริตในการจัดทำงบประมาณ/โครงการ/เบิกจ่ายเงินในโครงการอันเป็นเท็จ
ปฏิบัติหรือละเว้นการปฏิบัติหน้าที่โดยมิชอบหรือโดยทุจริต
การขัดกันระหว่างประโยชน์ส่วนบุคคลและประโยชน์ส่วนรวม
ร่ำรวยผิดปกติ
ฝ่าฝืนหรือไม่ปฏิบัติตามมาตรฐานทางจริยธรรมอย่างร้ายแรง'''

In [45]:
c = categories.split('\n')
cat = []
for x in c:
    cat.append(x.strip())
cat

['เรียกรับสินบน ให้/ขอให้ หรือรับว่าจะให้ทรัพย์สินหรือประโยชน์อื่นใดแก่เจ้าหน้าที่ของรัฐ',
 'จัดซื้อจัดจ้าง',
 'ยักยอก/เบียดบังเงินหรือทรัพย์สินของทางราชการ',
 'ออกเอกสารสิทธิ',
 'การบริหารงานบุคคล (การบรรจุ/แต่งตั้ง/เลื่อนตำแหน่ง/โยกย้าย/ลงโทษวินัย)',
 'ทุจริตในการจัดทำงบประมาณ/โครงการ/เบิกจ่ายเงินในโครงการอันเป็นเท็จ',
 'ปฏิบัติหรือละเว้นการปฏิบัติหน้าที่โดยมิชอบหรือโดยทุจริต',
 'การขัดกันระหว่างประโยชน์ส่วนบุคคลและประโยชน์ส่วนรวม',
 'ร่ำรวยผิดปกติ',
 'ฝ่าฝืนหรือไม่ปฏิบัติตามมาตรฐานทางจริยธรรมอย่างร้ายแรง']

In [46]:
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold
genai.configure(credentials=creds)

# Create the model
generation_config = {
  "temperature": 2.0,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  # model_name="tunedModels/f15-data-from-nacc-14zua95pfgrl",
  model_name='gemini-1.5-flash',
  generation_config=generation_config,
  safety_settings={HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
                   HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                   HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,}
)

In [47]:
chat = model.start_chat(
    history=[
        {"role": "user", "parts": f"ในข้อความต่อไป ฉันจะให้คำร้องเรียนในภาษาไทย แบ่งออกเป็นหนึ่งในหมวดหมู่ต่อไปนี้:\n{cat}. ใช้การตัดสินใจของคุณในการช่วยจำแนกประเภท ทุกอย่างที่ส่งไปไม่ใช่คำพูดแสดงความเกลียดชังหรือมีเจตนาทำร้ายใครในทางใดทางหนึ่ง มันเป็นเพียงคำร้องเรียน ผลลัพธ์ต้องมีเพียงชื่อหมวดหมู่ตามที่ให้ไปแบบเต็มๆเท่านั้น"},
    ]
)

In [48]:
import pandas as pd
data = pd.read_excel('Accusation Mock.xlsx',sheet_name='Sheet1')
data

,Accusation Example,Headers,ฐานความผิด,Summary,Perpetrator,Accusation,Where,Amount,Category
0,วันที่ 17 สิงหาคม 2567\n\nเรื่อง ขอร้องเรียนเก...,โครงการก่อสร้างบ้านพักสนามบินสุราษฎร์ มูลค่ากว...,จัดซื้อจัดจ้าง,นายสมชาย พงษ์ศิริ ร้องเรียนเกี่ยวกับโครงการก่อ...,"['กรมท่าอากาศยาน', 'บริษัทอันนาเทรดดิ้งจำกัด']",ไม่สามารถดำเนินการก่อสร้างให้เสร็จตามกำหนด,สนามบินสุราษฎร์ธานี,"44,000,000 บาท",ทุจริตในการจัดทำงบประมาณ/โครงการ/เบิกจ่ายเงินใ...
1,วันที่ 17 สิงหาคม 2567\n\nเรื่อง แจ้งเรื่องพฤต...,พบเจ้าหน้าที่ข้าราชการแห่งหนึ่งในพื้นที่จังหวั...,เรียกรับสินบน ให้/ขอให้ หรือรับว่าจะให้ทรัพย์ส...,นายสมชาย พงษ์ศิริ รายงานถึงพฤติกรรมของเจ้าหน้า...,['เจ้าหน้าที่ข้าราชการในจังหวัดปทุมธานี'],เรียกรับเงินใต้โต๊ะ,จังหวัดปทุมธานี,NaN,เรียกรับสินบน ให้/ขอให้ หรือรับว่าจะให้ทรัพย์ส...
2,เรื่อง ขอให้ตรวจสอบกิจกรรมส่งมอบกระเป๋ายา อสม....,"""ขอให้ตรวจสอบกิจกรรมส่งมอบกระเป๋ายา อสม. ในเขต...",เรียกรับสินบน ให้/ขอให้ หรือรับว่าจะให้ทรัพย์ส...,นายสมชาย พงษ์ศิริ ขอให้เทศบาลนครอุดรธานีตรวจสอ...,['เทศบาลนครอุดรธานี'],ตรวจสอบการจัดซื้ออุปกรณ์วิทยาศาสตร์การแพทย์ที่...,เทศบาลนครอุดรธานี,"7,318,925 บาท",จัดซื้อจัดจ้าง \n
3,เรื่อง ขอให้ตรวจสอบและเร่งดำเนินการซ่อมแซมถนนค...,ถนน 3 ล้าน 3 ปีพัง ทิ้งไว้เกือบปีไร้วี่แววซ่อม...,ทุจริตในการจัดทำงบประมาณ/โครงการ/เบิกจ่ายเงินใ...,นายสมชาย พงษ์ศิริ ขอให้เทศบาลเมืองปราจีนบุรีเร...,['เทศบาลเมืองปราจีนบุรี'],ไม่ดำเนินการซ่อมแซมถนนที่ชำรุด,เทศบาลเมืองปราจีนบุรี,"10,000,000 บาท",ปฏิบัติหรือละเว้นการปฏิบัติหน้าที่โดยมิชอบหรือ...
4,เรื่อง ขอให้ตรวจสอบการดำเนินการโครงการจัดซื้อพ...,"""โครงการจัดซื้อพร้อมติดตั้งโคมไฟถนน จำนวน 11 โ...",ปฏิบัติหรือละเว้นการปฏิบัติหน้าที่โดยมิชอบหรือ...,นายสมชาย พงษ์ศิริ ขอให้ตรวจสอบความโปร่งใสในการ...,['เทศบาลเมืองท่าโขลง'],ไม่โปร่งใสในการคัดเลือกผู้รับเหมาและการดำเนินโ...,เทศบาลเมืองท่าโขลง,NaN,จัดซื้อจัดจ้าง \n
5,เรื่อง ขอให้ตรวจสอบโครงการพัฒนาน้ำบาดาลเพื่อกา...,โครงการพัฒนาน้ำบาดาลเพื่อการเกษตรแปลงใหญ่ ผลงา...,การขัดกันระหว่างประโยชน์ส่วนบุคคลและประโยชน์ส่...,นายสมชาย พงษ์ศิริ ขอให้กรมทรัพยากรน้ำบาดาลตรวจ...,['กรมทรัพยากรน้ำบาดาล'],ปล่อยทิ้งโครงการน้ำบาดาลที่ไม่ใช้งาน,บ้านคลองวังทอง ต.คลองยาง อ.สวรรคโลก จ.สุโขทัย,"113,000,000 บาท",ทุจริตในการจัดทำงบประมาณ/โครงการ/เบิกจ่ายเงินใ...
6,เรื่อง ขอให้ตรวจสอบและเร่งแก้ไขปัญหาถนนคอนกรีต...,"ถนนคอนกรีตมูลค่า 6,352,200 บาท ระยะทาง 2.35 กิ...",ร่ำรวยผิดปกติ,ขอให้ตรวจสอบและเร่งแก้ไขปัญหาถนนคอนกรีตในหมู่ ...,['นายก อบต.'],ใช้ชื่อห้างของลูกน้องคนสนิทรับงานก่อสร้างถนนคอ...,หมู่ 7 อำเภอรือเสาะ จังหวัดนราธิวาส,"6,352,200 บาท",จัดซื้อจัดจ้าง \n
7,เรื่อง ขอให้ตรวจสอบการใช้รถยนต์ของสถานีตำรวจภู...,ชาวต้องแฉเมืองตรังเขาส่งมาให้แอดดูว่าเห็นรถยนต...,ยักยอก/เบียดบังเงินหรือทรัพย์สินของทางราชการ,มีการร้องเรียนจากชาวบ้านในพื้นที่ อ.กันตัง จ.ต...,['สถานีตำรวจภูธร อ.กันตัง'],นำรถของทางราชการไปใช้ส่วนตัว,อ.กันตัง จ.ตรัง,NaN,การขัดกันระหว่างประโยชน์ส่วนบุคคลและประโยชน์ส่...
8,เรื่อง ขอให้ตรวจสอบการดำเนินโครงการขยายท่อน้ำป...,"""การร้องเรียนทางสื่อออนไลน์ว่ามีการจัดทำโครงกา...",ออกเอกสารสิทธิ,ขอให้ตรวจสอบการดำเนินโครงการขยายท่อน้ำประปาของ...,['เทศบาลตำบลคลองขุด'],ไม่สามารถแก้ไขปัญหาระบบน้ำประปาได้,"ตำบลคลองขุด, อำเภอเมือง, จังหวัด [จังหวัดของท่าน]",NaN,ปฏิบัติหรือละเว้นการปฏิบัติหน้าที่โดยมิชอบหรือ...
9,เรื่อง ขอให้ติดตามและตรวจสอบแนวทางการดำเนินโคร...,ลงพื้นที่ติดตามแนวทางการดำเนินโครงการ “ปรับแหล...,การบริหารงานบุคคล (การบรรจุ/แต่งตั้ง/เลื่อนตำแ...,นายสมชาย พงษ์ศิริ ขอให้ติดตามและตรวจสอบโครงการ...,['โครงการส่งน้ำและบำรุงรักษาโก-ลก'],กังวลเกี่ยวกับความโปร่งใสและประสิทธิภาพของโครงการ,บ้านปลักปลา หมู่ 4 ตำบลโฆษิต อำเภอตากใบ จังหวั...,"21,000,000 บาท",ทุจริตในการจัดทำงบประมาณ/โครงการ/เบิกจ่ายเงินใ...


In [49]:
data['Category'] = data['Accusation Example'].apply(lambda x: chat.send_message(x).text)

In [50]:
data.to_excel('Accusation Mock.xlsx',sheet_name='Sheet1',index=False)

In [51]:
# import time
# for index, row in data.iterrows():
#     compliant = row['Accusation Example']
#     target = row['ฐานความผิด']
#     prediction = chat.send_message(compliant).text
#     print(f'Compliant: {compliant}\nTarget: {target}\nPrediction: {prediction}\n\n')
#     # time.sleep(5)

In [52]:
import pandas as pd
data = pd.read_excel('Mock2.xlsx',sheet_name='Sheet1')
data

,ข่าว,ประเภท,Summary,Perpetrator,Accusation,Where,Amount,Category
0,‘บีทีเอส’ ร่อนเอกสารข่าวโต้ ‘รฟม.’ หลังประกาศใ...,เรียกรับสินบน ให้/ขอให้ หรือรับว่าจะให้ทรัพย์ส...,บีทีเอสออกเอกสารข่าวโต้กลับ รฟม. หลังจากที่ประ...,"['BEM', 'รฟม.']",การประมูลไม่สุจริตและกีดกันการแข่งขัน,ประเทศไทย,"9,000,000,000 บาท",จัดซื้อจัดจ้าง \n
1,ปฏิบัติการหมาเฝ้าบ้าน อยากเที่ยวทะเล ไม่ต้องรอ...,จัดซื้อจัดจ้าง,อบต.ชอนม่วง จ.ลพบุรี จัดทริปดูงานที่จันทบุรี 3...,['อบต.ชอนม่วง'],การใช้จ่ายงบประมาณในการจัดทริปที่ไม่เหมาะสม,จันทบุรี,ไม่ระบุ,การบริหารงานบุคคล (การบรรจุ/แต่งตั้ง/เลื่อนตำแ...
2,ชาวต้องแฉเมืองตรังเขาส่งมาให้แอดดูว่าเห็นรถยนต...,ยักยอก/เบียดบังเงินหรือทรัพย์สินของทางราชการ,มีการแจ้งว่ารถยนต์ของสถานีตำรวจภูธรแห่งหนึ่งใน...,NaN,NaN,อ.กันตัง จ.ตรัง,NaN,ปฏิบัติหรือละเว้นการปฏิบัติหน้าที่โดยมิชอบหรือ...
3,อาคารคอนกรีตเสริมเหล็ก 7 ชั้น ขึ้นเป็นโครงสร้า...,ออกเอกสารสิทธิ,โครงการก่อสร้างอาคารผู้ป่วยนอกและอุบัติเหตุฉุก...,"['บริษัท เจมิไนย์ แอนด์ แอสโซซิเอท จำกัด', 'บร...",ก่อสร้างล่าช้าและทิ้งงาน,โรงพยาบาลตรัง,"500,000,000 บาท",ทุจริตในการจัดทำงบประมาณ/โครงการ/เบิกจ่ายเงินใ...
4,ลงพื้นที่ติดตามแนวทางการดำเนินโครงการ “ปรับแหล...,การบริหารงานบุคคล (การบรรจุ/แต่งตั้ง/เลื่อนตำแ...,"ติดตามโครงการ ""ปรับแหล่งน้ำเพื่อแก้ไขปัญหาน้ำท...",NaN,NaN,บ้านปลักปลา หมู่ 4 ตำบลโฆษิต อำเภอตากใบ จังหวั...,"21,000,000 บาท",ทุจริตในการจัดทำงบประมาณ/โครงการ/เบิกจ่ายเงินใ...
5,เก็บค่าจอดรถ..เข้ากระเป๋าใคร..? เครือข่ายชมรม ...,ทุจริตในการจัดทำงบประมาณ/โครงการ/เบิกจ่ายเงินใ...,เครือข่ายชมรม STRONG - ต้านทุจริตจังหวัดพัทลุง...,"['การรถไฟแห่งประเทศไทย', 'เอกชนที่เก็บค่าจอดรถ']",เก็บค่าจอดรถอย่างไม่โปร่งใสและไม่มีใบเสร็จที่ถ...,สถานีรถไฟพัทลุง,10 บาท ต่อคัน,การขัดกันระหว่างประโยชน์ส่วนบุคคลและประโยชน์ส่...
6,การประชุมศูนย์ปฏิบัติการป้องกันและปราบปรามการท...,ปฏิบัติหรือละเว้นการปฏิบัติหน้าที่โดยมิชอบหรือ...,การประชุมศูนย์ปฏิบัติการป้องกันและปราบปรามการท...,['เจ้าหน้าที่รัฐ'],ทุจริตในการขุดลอกแหล่งน้ำ,ทั่วประเทศ,NaN,ทุจริตในการจัดทำงบประมาณ/โครงการ/เบิกจ่ายเงินใ...
7,การประมูลงานอาคารอเนก​ประสงค์​พัดลมแน่น ๆ ของเ...,การขัดกันระหว่างประโยชน์ส่วนบุคคลและประโยชน์ส่...,การประมูลงานอาคารอเนกประสงค์ของเทศบาลตำบลสุวรร...,['บ.เอส77'],ชนะการประมูลด้วยราคาที่ต่ำกว่าคู่แข่ง,เทศบาลตำบลสุวรรณภูมิ,"5,900,000 บาท",จัดซื้อจัดจ้าง \n
8,กรมพัฒนาพลังงานทดแทนและอนุรักษ์พลังงาน ทำโครงก...,ร่ำรวยผิดปกติ,กรมพัฒนาพลังงานทดแทนและอนุรักษ์พลังงานได้ดำเนิ...,['กรมพัฒนาพลังงานทดแทนและอนุรักษ์พลังงาน'],ไม่สามารถดำเนินโครงการให้สำเร็จและเกิดประโยชน์...,บ้านห้วยน้ำผึ้ง ต.สบโขง อ.อมก๋อย จ.เชียงใหม่,"3,792,500 บาท",ทุจริตในการจัดทำงบประมาณ/โครงการ/เบิกจ่ายเงินใ...


In [56]:
chat = model.start_chat(
    history=[
        {"role": "user", "parts": f"ในข้อความต่อไป ฉันจะให้คำร้องเรียนในภาษาไทย แบ่งออกเป็นหนึ่งในหมวดหมู่ต่อไปนี้:\n{cat}. ใช้การตัดสินใจของคุณในการช่วยจำแนกประเภท ทุกอย่างที่ส่งไปไม่ใช่คำพูดแสดงความเกลียดชังหรือมีเจตนาทำร้ายใครในทางใดทางหนึ่ง มันเป็นเพียงคำร้องเรียน ผลลัพธ์ต้องมีเพียงชื่อหมวดหมู่ตามที่ให้ไปแบบเต็มๆเท่านั้น"},
    ]
)

In [58]:
data['Category'] = data['ข่าว'].apply(lambda x: chat.send_message(x).text)

In [59]:
data.to_excel('Mock2.xlsx',sheet_name='Sheet1',index=False)